In [5]:
!pip install PyMuPDF openai pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 58.8 MB/s eta 0:00:00


In [6]:
import fitz  # PyMuPDF
import os
import pandas as pd
import openai


In [8]:
!pip install fpdf
from fpdf import FPDF
import os # Import os module if not already imported

def create_resume(filename, content):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    for line in content.split('\n'):
        # Add a check for line content and potentially handle encoding issues if they arise
        # For simplicity, we assume basic text content is provided
        try:
            pdf.cell(200, 10, txt=line, ln=True)
        except Exception as e:
            print(f"Error adding line to PDF: {e}")
            print(f"Problematic line: {line}")
            # You might want to skip the line or handle the error differently
            pass
    pdf.output(filename)

sample_resumes = {
    "resume_ds.pdf": "Name: Asha\nSkills: Python, Machine Learning, Pandas, Deep Learning\nExperience: 3 years in Data Science at Infosys",
    "resume_dev.pdf": "Name: Rahul\nSkills: Java, Spring Boot, REST APIs, Docker\nExperience: 2 years in Software Development at TCS",
    "resume_hr.pdf": "Name: Sneha\nSkills: Recruitment, Onboarding, HR Policies\nExperience: 4 years in HR at Wipro",
}

# Define the directory where resumes will be saved
resume_folder = "sample_resumes"

# Create the directory if it doesn't exist
if not os.path.exists(resume_folder):
    os.makedirs(resume_folder)

for file, content in sample_resumes.items():
    path = os.path.join(resume_folder, file)
    create_resume(path, content)

print("Sample resume PDFs created.")

Sample resume PDFs created.


In [11]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set path where resumes are saved
resume_folder = "/content/drive/MyDrive/resumes"

# Set OpenAI API key (replace with your key)
openai.api_key = "your_openai_api_key"


Mounted at /content/drive


In [15]:
from IPython import get_ipython
from IPython.display import display
!pip install PyMuPDF openai pandas
import fitz  # PyMuPDF
import os
import pandas as pd
import openai
!pip install fpdf
from fpdf import FPDF
import os

def create_resume(filename, content):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    for line in content.split('\n'):
        try:
            pdf.cell(200, 10, txt=line, ln=True)
        except Exception as e:
            print(f"Error adding line to PDF: {e}")
            print(f"Problematic line: {line}")
            pass
    pdf.output(filename)

sample_resumes = {
    "resume_ds.pdf": "Name: Asha\nSkills: Python, Machine Learning, Pandas, Deep Learning\nExperience: 3 years in Data Science at Infosys",
    "resume_dev.pdf": "Name: Rahul\nSkills: Java, Spring Boot, REST APIs, Docker\nExperience: 2 years in Software Development at TCS",
    "resume_hr.pdf": "Name: Sneha\nSkills: Recruitment, Onboarding, HR Policies\nExperience: 4 years in HR at Wipro",
}

resume_folder = "sample_resumes"

if not os.path.exists(resume_folder):
    os.makedirs(resume_folder)

for file, content in sample_resumes.items():
    path = os.path.join(resume_folder, file)
    create_resume(path, content)

print("Sample resume PDFs created.")
from google.colab import drive
drive.mount('/content/drive')

resume_folder_local = "sample_resumes"

openai.api_key = "your_openai_api_key"

def extract_text_from_pdfs(folder_path):
    data = []
    if not os.path.exists(folder_path):
        print(f"Directory not found: {folder_path}")
        return pd.DataFrame(data)

    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            file_path = os.path.join(folder_path, file)
            try:
                doc = fitz.open(file_path)
                text = ""
                for page in doc:
                    text += page.get_text()
                data.append({'filename': file, 'content': text})
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
                pass

    return pd.DataFrame(data)

df = extract_text_from_pdfs(resume_folder_local)
df.head()

Sample resume PDFs created.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,filename,content
0,resume_hr.pdf,"Name: Sneha\nSkills: Recruitment, Onboarding, ..."
1,resume_ds.pdf,"Name: Asha\nSkills: Python, Machine Learning, ..."
2,resume_dev.pdf,"Name: Rahul\nSkills: Java, Spring Boot, REST A..."


In [16]:
def classify_resume(resume_text):
    prompt = f"""
You are a resume classification expert. Based on the following resume content, classify the candidate into one of the following categories:
- Data Science
- Software Development
- Human Resources (HR)
- Marketing
- Finance

Resume Content:
{resume_text[:3000]}

Respond with only the category name.
"""
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return response.choices[0].message['content'].strip()
    except Exception as e:
        return f"Error: {str(e)}"


In [17]:
# Apply the classification function to each resume
df['predicted_category'] = df['content'].apply(classify_resume)

# Show preview
df[['filename', 'predicted_category']].head()


,filename,predicted_category
0,resume_hr.pdf,Error: \n\nYou tried to access openai.ChatComp...
1,resume_ds.pdf,Error: \n\nYou tried to access openai.ChatComp...
2,resume_dev.pdf,Error: \n\nYou tried to access openai.ChatComp...


In [20]:
# Set OpenAI API key (replace with your actual key)
# It's highly recommended to use environment variables or a secure method
# to store your API key instead of hardcoding it.
openai.api_key = "YOUR_ACTUAL_OPENAI_API_KEY"

# The classify_resume function remains the same
def classify_resume(resume_text):
    prompt = f"""
You are a resume classification expert. Based on the following resume content, classify the candidate into one of the following categories:
- Data Science
- Software Development
- Human Resources (HR)
- Marketing
- Finance

Resume Content:
{resume_text[:3000]}

Respond with only the category name.
"""
    try:
        # Ensure the API key is set before this call (handled above)
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return response.choices[0].message['content'].strip()
    except Exception as e:
        return f"Error: {str(e)}"

# Apply the classification function to each resume
df['predicted_category'] = df['content'].apply(classify_resume)

# Show preview
df[['filename', 'predicted_category']].head()

,filename,predicted_category
0,resume_hr.pdf,Error: \n\nYou tried to access openai.ChatComp...
1,resume_ds.pdf,Error: \n\nYou tried to access openai.ChatComp...
2,resume_dev.pdf,Error: \n\nYou tried to access openai.ChatComp...


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('filename').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [21]:
output_csv_path = "/content/drive/MyDrive/classified_resumes.csv"

# Save to CSV
df[['filename', 'predicted_category']].to_csv(output_csv_path, index=False)

print(f"✅ Classification complete. Output saved to:\n{output_csv_path}")


✅ Classification complete. Output saved to:
/content/drive/MyDrive/classified_resumes.csv
